# Introduction to Python and Artificial Intelligence Through Pathfinding Algorithms


**Howell Dalton "Chip" McCullough IV**

Artificial Intelligence describes an application of Computer Science that seeks to mimic the human brain. Historically, intellegent systems are defined to act in usually one of four categories (Russel & Norving, 2010):

  * "thinking humanly"
  * "thinking rationally"
  * "acting humanly"
  * "acting rationally"

In introducing Artificial Intelligence, we will concern ourselves with pathfinding algorithms -- algorithms in which an intelligent agent seeks to find an ideal outcome from point A to point B. Particularly, we will be developing intellegent agents that will "act rationally", as the best path from point A from Point B may not be known, let alone exist. In this project, we will be using Python 3 (which should already be installed on your machine), and the default GUI library Tkinter.

## Modeling a Maze

First things first, we need to construct a model for our intelligent agent to traverse. Because we want real solutions, we need to build a maze under the following parameters:

  1. The maze is perfect (i.e. solvable)
  2. The maze is randomly generated (so that we cannot hard code a solution)

### Graphs, Vertices, and Edges

Finally we're about to begin writing code, but before we get into that, we need to go through a bit of a mathematical background. We will be creating our maze by creating a randomly generated spanning tree. 

So, what's a tree? What does it mean for a tree to be a spanning tree? 

Followng the chain of jargon, a tree is simply a graph -- a mathematical collection of vertices, $V$, and edges, $E$, defined as $\mathcal{G} = (V, E)$. Specifically a tree is a graph in which all vertices have exactly one "parent" and may have up ton $n$ children. A vertex $v \in V$ defines a singular point in a graph that may or may not be connected to another vertex by an edge, $e \in E$.

Finally, a spanning tree is a tree constructed from a graph in which all edges are connected with a minimum number of edges, and the graph is acyclic (does not contain any cycles).

With that, we will begin by creating a model for a vertex, the Node:

```python
import binascii
import os

class Node(object):
    """
    Node data structure
    """
    def __init__(self):
        self.__north = None
        self.__south = None
        self.__east = None
        self.__west = None
        self.__degree = 0
        self.__visited = 0
        self.__key = binascii.hexlify(os.urandom(32))

    @property
    def north(self):
        return self.__north

    @north.setter
    def north(self, north):
        self.__north = north

    @property
    def south(self):
        return self.__south

    @south.setter
    def south(self, south):
        self.__south = south

    @property
    def east(self):
        return self.__east

    @east.setter
    def east(self, east):
        self.__east = east

    @property
    def west(self):
        return self.__west

    @west.setter
    def west(self, west):
        self.__west = west

    @property
    def degree(self) -> int:
        return self.__degree

    @degree.setter
    def degree(self, deg: int):
        self.__degree = deg

    @property
    def visited(self):
        return self.__visited

    @visited.setter
    def visited(self, mark: int):
        self.__visited = mark

    @property
    def key(self):
        return self.__key

    def isNorthOf(self, node):
        return self.__south is node

    def isSouthOf(self, node):
        return self.__north is node

    def isEastOf(self, node):
        return self.__west is node

    def isWestOf(self, node):
        return self.__east is node

    def __eq__(self, other) -> bool:
        return self.key == other.key

    def __str__(self) -> str:
        return str(self.__visited)
```

## Depth First Search

## Breadth First Search

# References

Russell, S. J., & Norving, P. (2010). Introduction. In Artificial intelligence: A modern approach (pp. 1-33). New Jersey: Pearson. 